In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import matplotlib

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
import numpy as np
import pandas as pd
from decimal import Decimal
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import stats, interpolate
import scipy
import warnings
import datetime as dt
import landau
from multiprocessing import Pool, cpu_count

from matplotlib.patches import Rectangle

import importlib
import os
from tqdm.auto import tqdm

# local imports
from lib.constants import *

from scipy.integrate import quad, tplquad
from scipy.special import erf
from scipy.interpolate import CubicSpline

In [ ]:
dosave = True
plttitle = "ICARUS Run 1"
plotqual="run1_"
savedir = "./plots_9_13_23/%s" % plotqual
plt.rcParams.update({'font.size': 14})
datadir = "/pnfs/sbn/persistent/users/gputnam/calib-data/outputs/"
# datadir = "/icarus/data/users/gputnam/MCP2022G/mc/cosmic3/"

isMC = False
dataf = datadir + "P2_scaleYZ_Run1.txt"

In [ ]:
yz_ybin = np.linspace(-180, 130, 32)
yz_ylos = yz_ybin[:-1]
yz_yhis = yz_ybin[1:]
yz_ys = (yz_ylos + yz_yhis) / 2.

yz_zbin = np.linspace(-900, 900, 181)
yz_zlos = yz_zbin[:-1]
yz_zhis = yz_zbin[1:]
yz_zs = (yz_zlos + yz_zhis) / 2.

In [ ]:
tpcs = ["EE", "EW", "WE", "WW"]

In [ ]:
zy_mpvs = {}
zy_mpverrs = {}
zy_Ns = {}

for tpc in tpcs:
    zy_mpvs[tpc] = []
    zy_mpverrs[tpc] = []
    zy_Ns[tpc] = []
    for _ in yz_ys:
        zy_mpvs[tpc].append([])
        zy_mpverrs[tpc].append([])
        zy_Ns[tpc].append([])
        
        for _ in yz_zs:
            zy_mpvs[tpc][-1].append(-1)
            zy_mpverrs[tpc][-1].append(-1)
            zy_Ns[tpc][-1].append(-1)

In [ ]:
with open(dataf) as f:
    next(f)
    for line in f:
        dat = line.rstrip("\n").split("\t")
        tpc = dat[0]
        iy = int(dat[1])
        iz = int(dat[2])
        mpv = float(dat[4])
        merr = float(dat[5])
        N = int(float(dat[6]))
        
        zy_mpvs[tpc][iy][iz] = mpv
        zy_mpverrs[tpc][iy][iz] = merr
        zy_Ns[tpc][iy][iz] = N
        

In [ ]:
diffs = []

for tpc in tpcs:
    for iy in range(1, len(zy_mpvs[tpc])-1):
        for iz in range(1, len(zy_mpvs[tpc][iy])-1):
            this_mpv = zy_mpvs[tpc][iy][iz]
            mean_diff = sum([
                zy_mpvs[tpc][iy-1][iz],
                zy_mpvs[tpc][iy+1][iz],
                zy_mpvs[tpc][iy][iz-1],
                zy_mpvs[tpc][iy][iz+1],
            ])/4/this_mpv - 1
            diffs.append(mean_diff)

In [ ]:
def gaus(x, A, mu, sigma):
    return A*np.exp(-(x-mu)**2/(2*sigma**2))

In [ ]:
bins = np.linspace(-0.05, 0.05, 21)
N,bins,_ = plt.hist(diffs, bins=bins)
centers = (bins[1:] + bins[:-1]) / 2.

popt, perr = curve_fit(gaus, centers, N, sigma=np.sqrt(N))

pltcenter = np.linspace(centers[0], centers[-1], 1000)
plt.plot(pltcenter, gaus(pltcenter, *popt))


plt.xlabel("Spatial Variation in Y-Z Scale Factors")
plt.ylabel("# Bins")
plt.text(0.05, 0.9, 
         "$\\sigma$: %.1f%%" % (popt[-1]*100), 
         transform=plt.gca().transAxes, fontsize=14)

plt.title(plttitle)

if dosave:
    plt.savefig(savedir + "spatial_variation_yz.pdf")
popt[-1]*100